# Problem Definition

---

If we have 1 million vaccine doses available and we have to distribute the vaccine by county (with average country population of 100,000), we have to find 10 counties with the highest risk to send the vaccines to. In order to do so, we want to find maxima value of the 10 counties in question. 

Fitness Function

$\sum_{i=1}^g n_i R_t (c+1)(h+1)d_i$

where: 

$g = $ number of age groups from Census Bureau

$n_i = $ population size of each age group

$R_t = $ infection rate OR reproductive number

$c = $ case density in the county risk factor

$h = $ ICU headroom risk factor

$d_i = $ likelihood of death for each age group (modified to fit CDC age bins)

For additional information on problem definition, please visit https://github.com/aungnay/vacdist#problemdefinition



# Library Import Section

In [ ]:
# installation of DEAP
!pip install deap

In [ ]:
# importing all the libraries needed for generating random values, data analysis, 
# n-dimension arrays, data visualization, plotting, and genetic algorithms
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from deap import algorithms, base, creator, tools
import timeit

# Data Import Section

Importing 
*   2019 census bureau's estimate data from all the counties in 50 states and District of Columbia with population data for different age groups
*   Real-time COVID-19 data from CovidActNow

Manually create an array with CDC data and variables for vaccine quantity and average county sizes

In [ ]:
# defining the columns that we want to read into the data frame
col = ['STATE','COUNTY','STNAME','CTYNAME','YEAR','POPESTIMATE','AGE04_TOT','AGE59_TOT','AGE1014_TOT','AGE1519_TOT','AGE2024_TOT','AGE2529_TOT','AGE3034_TOT','AGE3539_TOT','AGE4044_TOT','AGE4549_TOT','AGE5054_TOT','AGE5559_TOT','AGE6064_TOT','AGE6569_TOT','AGE7074_TOT','AGE7579_TOT','AGE8084_TOT','AGE85PLUS_TOT']

# creating an empty data frame with the headers
states = pd.DataFrame(columns = col)

# reads in 2019 census data csv files from the Census Bureau using the FIPS state values 
# https://www.census.gov/content/census/en/data/datasets/time-series/demo/popest/2010s-counties-detail.html
id = 1
for i in range(0,56):
  if id < 10:
    code = '0' + str(id)
  else:
    code = str(id)
  # excluding non-states and New Mexico but include District of Columbia
  # New Mexico (35) csv file was having problems with the pandas read
  if code not in ['03','07','14','35','43','52']:
    # reading in only the columns defined above and defining state and county
    # data types as strings so that we can concatenate them later
    temp = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/cc-est2019-agesex-'+code+'.csv', usecols = col, dtype={'STATE':str,'COUNTY':str})
    # the Census Bureau splits the FIPS values into two columns of state and county
    # we are concatenating them and creating a new column call FIPS
    temp['FIPS'] = temp['STATE'] + temp['COUNTY']
    # slicing the data frame for 2019 data
    temp = temp[temp['YEAR'] == 12]
    # concatenating state data frames into a single states data frame
    states = pd.concat([states,temp])
  id += 1

In [ ]:
# check data frame for number of counties for debugging
# print(states)

Adding New Mexico. New Mexico has a county named "Doña Ana County". ñ was having problems with utf-8. ñ was changed to n and the csv file was uploaded to github and read. Then concatenated to states.

In [ ]:
nm = pd.read_csv('https://raw.githubusercontent.com/aungnay/vacdist/main/data/cc-est2019-agesex-35.csv', dtype={'STATE':str,'COUNTY':str})
nm['FIPS'] = nm['STATE'] + nm['COUNTY']
states = pd.concat([states,nm])

In [ ]:
# check data frame for number of counties for debugging
# print(states)

Importing COVID-19 Reproductive Rates for all counties. (Not all counties have data) from COVID Act Now.

In [ ]:
# reading in only the FIPS and real-time Covid reproductive rate and defining 
# FIPS column as a string so that we can use it to merge the data frames
# rRate = pd.read_csv('https://api.covidactnow.org/v2/counties.csv?apiKey=98adfe7ece9840a8a8f4af1c0dfea76c', usecols = ['fips','metrics.infectionRate','riskLevels.caseDensity','riskLevels.icuHeadroomRatio'], dtype={'fips':str})

# reading in only the FIPS and Covid reproductive rate at 14:09 on 11/30/2020 
# and defining FIPS column as a string so that we can use it to merge the data 
# frames. this allows for independent calculation of optimal solution and do
# comparisons against the GA with different parameters
rRate = pd.read_csv('https://raw.githubusercontent.com/aungnay/vacdist/main/data/counties2011301409.csv', usecols = ['fips','metrics.infectionRate','riskLevels.caseDensity','riskLevels.icuHeadroomRatio'], dtype={'fips':str})

# renaming column name to have exact match with the states data frame
rRate.rename(columns={'fips':'FIPS'}, inplace=True)

In [ ]:
# check data frame for number of counties for debugging
# print(rRate)

Inner Join of two data frames above

In [ ]:
# inner joining with merge function to merge the states and rRate data frames
# using FIPS as key column. only merging rows with matching data on both data 
# frames. the NaN data for reproductive rates are still in the new data frame
cdf = pd.merge(states, rRate, on=['FIPS'])

# check data frame for number of counties after merge for debugging
print(cdf)

In [ ]:
# drops all counties with NaN
cdfn = cdf.dropna()

# check data frame for number counties after drop for debugging
print(cdfn)

# exporting cdfn.csv


In [ ]:
# exporting cdfn data frame to my google drive
#from google.colab import drive
#drive.mount('drive')
#cdfn.to_csv('cdfn.csv')
#!cp cdfn.csv "drive/My Drive/"

In [ ]:
# modified death risk array
d = [1/9,1/16,1/16,1/16,1,1,4,4,10,10,30,30,30,90,90,220,220,630]

# vaccine quantity
v_qty = 1000000

# mean county population size
c_size = 100000

# ga population size
n_sol = len(cdfn) * 0.01

# Genetic Algorithm Section

In [ ]:
# creates the new class FitnessMax that inherits from base.Fitness with weight of 1.0 for maximizing function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
# generate solutions from the range of index of the dataframe without replacement
# so that counties would not be repeated. the population of solutions are 
# created as a list
toolbox = base.Toolbox()
toolbox.register("index", np.random.choice, len(cdfn), int(v_qty/c_size), replace=False)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.index)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# test generation of a single solution for debugging
toolbox.individual()

In [ ]:
# test generation of a population for debugging
toolbox.population(n=31)

In [ ]:
# fitness function as shown in the problem description
def risk(individual):
  county_risk = 0
  # checking if the solution has a repeat of counties through mutation or crossover
  if len(individual) != len(set(individual)):
    return county_risk,
  # calculates fitness function for the solution
  else:
    for i in individual:
      for j in range(6,24):
        temp = cdfn.iloc[i][j]*cdfn.iloc[i][25]*(cdfn.iloc[i][26]+1)*(cdfn.iloc[i][27]+1)*d[j-6]
        county_risk += temp
    return county_risk,

In [ ]:
# registration of operators for the DEAP toolbox
# evaluate function will use the my fitness function "risk"
toolbox.register("evaluate", risk)
# crossover function will be the DEAP one point crossover method
# other crossover methods are tools.cxTwoPoint, tools.cxOrdered, tools.csUniform,
# and tools.cxPartialyMatched
toolbox.register("mate", tools.cxOnePoint)
# mutate function will be the DEAP uniform mutation with the probabilty of 0.1
# with lower bound of 0 and upper bound of length of cdfn data frame -1, based 
# on number of rows in my data frame
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(cdfn)-1, indpb=1/len(cdfn))
# selection fundtion will be the DEAP roulette method with default settings
# other selection methods are tools.selTournament and tools.selBest
toolbox.register("select", tools.selRoulette)

In [ ]:
def ga():
  # timer start
  tic = timeit.default_timer()
  
  random.seed(64)
  # number of solutions for the population or population size
  n_sol = 300
  # generating the population using the toolbox and population size
  pop = toolbox.population(n=n_sol)
  # DEAP hall of fame keeps defined number of best solutions in the population
  hof = tools.HallOfFame(0)
  # defining what should be included in the statistics
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)
  # generation of new population using DEAP simple
  # to add hall of fame, change hof value above and add "halloffame=hof" as parameter
  # set cxpb to 0 to not use crossover. set cxpb between 0.6 and 0.9
  # set mutpb to 0 to not use mutation. set mutpb between 1/len(individual) and 1/pop
  pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.75, mutpb=0.1, ngen=100, stats=stats, verbose=True)
  
  # timer stop
  toc = timeit.default_timer()

  # print elapsed time
  print(toc - tic)

  # extract statistics:
  minFitnessValues, meanFitnessValues, maxFitnessValues = log.select("min", "avg", "max")

  # plot statistics:
  sns.set_style("whitegrid")
  plt.plot(minFitnessValues, color='blue')
  plt.plot(meanFitnessValues, color='red')
  plt.plot(maxFitnessValues, color='green')
  plt.xlabel('Generation')
  plt.ylabel('Min / Average / Max Fitness')
  plt.title('Min, Average, and Max fitness over Generations')
  plt.show() 

  # returns the list of top 5 solutions
  return tools.selBest(pop, k=5)

In [ ]:
# runs the ga method and sets the best 5 solutions as a numpy array
best = ga()

In [ ]:
# prints the list of top 5 solutions with the name of counties
for i in range(0,5):
  print("set"+str(i))
  print(risk(best[0]))
  for j in range(0,10):
    print(cdfn.iloc[best[i][j]][3], ", ", cdfn.iloc[best[i][j]][2])
  print()

#Findings

For detailed findings, please visit https://github.com/aungnay/vacdist#findings

# References

Alajmi, A., & Wright, J. (2014, June). Selecting the most efficient genetic algorithm sets in solving unconstrained building optimization problem. International Journal of Sustainable Built Environment, 3(1), 18-26.

Bäck, T. (2000). Introduction to evolutionary algorithms. In Evolutionary Computation 1 Basic Algorithms and Operators. Bristol, United Kingdom: Institute of Physics.

Blickle, T., & Thiele, L. (1995, July). A Mathematical Analysis of Tournament Selection. Proceedings of the 6th International Conference on Genetic Algorithms (ICGA95), 9-16.

Centers for Disease Control and Prevention. (2020, August 18). COVID-19 Hospitalization and Death by Age. Retrieved from Centers for Disease Control and Prevention: https://www.cdc.gov/coronavirus/2019-ncov/covid-data/investigations-discovery/hospitalization-death-by-age.html

CovidActNow. (2020). Tools. Retrieved from CovidActNow: https://covidactnow.org/tools

Gad, A. (2020). GeneticAlgorithmPython. Retrieved from GitHub: https://github.com/ahmedfgad/GeneticAlgorithmPython

Haupt, R. L., & Haupt, S. E. (2004). Practical Genetic Algorithms. Hoboken, New Jersey: John Wiley & Sons.

McGowan, C. (2017, March 9). Genetic Algorithms: Tournament Selection. Retrieved from Medium: https://medium.com/@c4lv1nmcg0wan/genetic-algorithms-tournament-selection-b150bc76f0d8

Patel, R., Longini, I. M., Jr, & Halloran, M. E. (2005). Finding optimal vaccination strategies for pandemic influenza using genetic algorithms. Journal of theoretical biology, 234(2), 201–212. https://doi.org/10.1016/j.jtbi.2004.11.032

Rainville, F.-M. d., Fortin, F.-A., Gagné, C., Gagnon, O., Gardner, M.-A., Grenier, S., . . . Parizeau, M. (2020). DEAP. Retrieved from GitHub: https://github.com/deap/deap

Song, C. (2019). Decision_Making_with_GA_using_DEAP. Retrieved from GitHub: https://github.com/daydrill/ga_pycon_2016_apac/blob/master/Decision_Making_with_GA_using_DEAP.ipynb

United States Census Bureau. (2020). County Population by Characteristics: 2010-2019. Retrieved from United States Census Bureau: https://www.census.gov/content/census/en/data/datasets/time-series/demo/popest/2010s-counties-detail.html
